In [ ]:
"""
This Notebook has snippets of scripts for each protein to calculate the pairwise distances between pairs
we detected as potential epistatic pairs.
In order to do that, one needs to first align the sequence to the sequence in the 3D structure to match the positions
then the pairwise distances can be taken.

At the end, a Wilcoxon test is ran comparing the pariwise distance distribution between the pairs we detected
and all pairwise distances. The Wilcoxon test is done with R.

So the general steps are:
1- Loading the structure that corresponds to the protein we are working on
2- Extracting the sequence from the protein and aligning and mapping sequence to residues in the structure
3- Loading the pairs with the significant p-value (we need their positions)
4- Calculating pairwise distances for these pairs and for all possible pairs in the structure
5- Wilcoxon test to check if the distances between the pairs we detected is significantly smaller than the total
distribution

Note: There will be repetetive codes in each snippet for each protein, I did not want to separate them in functions
or join them, to make each snippet separate, in case one wants to copy it for just one protein, or modefy it for
to be used on another protein
"""

In [ ]:
import sys
import os
import random
import pickle
import subprocess
from Bio.PDB import *
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Align import substitution_matrices
from pprint import pprint
import matplotlib.pyplot as plt
from copy import deepcopy
import scipy.stats as stats

In [ ]:
def wilcoxon_R(epi_distances, all_distances, current_seq):
    """
    This function outputs an R script to calculate Wilcoxon test
    runs the R script in a subprocess then deletes intermediate files and keeps the final value
    in tmp_wilcoxon.txt
    This script can be ran after running any of the previous snippets because it only needs the epi_distances dict
    and the all_distances dict
    file_name = current_seq.split(".")[0]
    """
    
    r_script = f"""
    # Read in the data
    options(warn=-1)

    distances_file <- scan("tmp.txt", what="", sep="\n")

    epi_distances = as.double(strsplit(distances_file[1], ", ")[[1]])
    all_distances = as.double(strsplit(distances_file[2], ", ")[[1]])

    w_test <- wilcox.test(epi_distances, all_distances, alternative="greater")
    print("These values are for {current_seq}")
    print(paste0("The welcox greater test statistic is: ", w_test$statistic, " and the p-value is: ", w_test$p.value))
    print(paste0("The AUC value is: ", w_test$statistic/(length(epi_distances) * length(all_distances))))
    
    w_test <- wilcox.test(epi_distances, all_distances, alternative="less")
    print("These values are for {current_seq}")
    print(paste0("The welcox less test statistic is: ", w_test$statistic, " and the p-value is: ", w_test$p.value))
    print(paste0("The AUC value is: ", w_test$statistic/(length(epi_distances) * length(all_distances))))
    """

    out_r_script =  open("tmp.r", "w")
    out_r_script.write(r_script)
    out_r_script.close()

    out_file = open("tmp.txt", "w")
    out_file.write(str(list(epi_distances.values()))[1:-1] + "\n")
    out_file.write(str(list(all_distances.values()))[1:-1] + "\n")
    out_file.close()
    # print(wilcoxon(all_epi_distances_sampled, all_distances_sampled, zero_method="wilcox"))
    subprocess.call(f"Rscript tmp.r > {current_seq}_wilcoxon.txt", shell=True)

    subprocess.call("rm tmp.r", shell=True)
    subprocess.call("rm tmp.txt", shell=True)

In [ ]:
blosum62 = substitution_matrices.load("BLOSUM62")
# The structures we used (bilogical assemblies pre-downloaded)
structures = ["../1NN2_N2.pdb1", "../1RUZ_H1.pdb1", "../2VIU_H3.pdb1", "../3BEQ_N1.pdb1", 
              "../6B0N_HIV_A_B.pdb1", "../6B0N_HIV_A_B.pdb1", "../6MYY_HIV1_subtype_c.pdb1"]

protein_letters_3to1 = {'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E',
'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S',
'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'}

# The sequences to align to the structures
my_seq_N2 = "MNPNQKIITIGSVSLTIATVCFLMQTAILVTTVTLHFKQYECDSPASNQVMPCEPIIIERNITEIVYLNNTTIEKEICPKVVEYRNWSKPQCQITGFAPFSKDNSIRLSAGGDIWVTREPYVSCDHGKCYQFALGQGTTLDNKHSNDTIHDRIPHRTLLMNELGVPFHLGTRQVCIAWSSSSCHDGKAWLHVCITGDDKNATASFIYDGRLVDSIGSWSQNILRTQESECVCINGTCTVVMTDGSASGRADTRILFIEEGKIVHISPLSGSAQHVEECSCYPRYPGVRCICRDNWKGSNRPVVDINMEDYSIDSSYVCSGLVGDTPRNDDRSSNSNCRNPNNERGNQGVKGWAFDNGDDVWMGRTISKDLRSGYETFKVIGGWSTPNSKSQINRQVIVDSDNRSGYSGIFSVEGKSCINRCFYVELIRGRKQEARVWWTSNSIVVFCGTSGTYGTGSWPDGANINFMPI"
my_seq_H1 = "MKVKLLVLLCTFTATYADTICIGYHANNSTDTVDTVLEKNVTVTHSVNLLEDSHNGKLCLLKGIAPLQLGNCSVAGWILGNPECELLISKESWSYIVETPNPENGTCYPGYFADYEELREQLSSVSSFERFEIFPKESSWPNHTVTGVSASCSHNGKSSFYRNLLWLTGKNGLYPNLSKSYANNKEKEVLVLWGVHHPPNIGDQRALYHTENAYVSVVSSHYSRRFTPEIAKRPKVRDQEGRINYYWTLLEPGDTIIFEANGNLIAPRYAFALSRGFGSGIITSNAPMDECDAKCQTPQGAINSSLPFQNVHPVTIGECPKYVRSAKLRMVTGLRNIPSIQSRGLFGAIAGFIEGGWTGMVDGWYGYHHQNEQGSGYAADQKSTQNAINGITNKVNSVIEKMNTQFTAVGKEFNKLERRMENLNKKVDDGFLDIWTYNAELLVLLENERTLDFHDSNVKNLYEKVKSQLKNNAKEIGNGCFEFYHKCNDECMESVKNGTYDYPKYSEESKLNREKIDGVKLESMGVYQILAIYSTVASSLVLLVSLGAISFWMCSNGSLQCRICI"
my_seq_H3 = "MKTIIALSYIFCLALGQDLPGNDNSTATLCLGHHAVPNGTLVKTITDDQIEVTNATELVQSSSTGKICNNPHRILDGIDCTLIDALLGDPHCDVFQNETWDLFVERSKAFSNCYPYDVPDYASLRSLVASSGTLEFITEGFTWTGVTQNGGSNACKRGPGSGFFSRLNWLTKSGSTYPVLNVTMPNNDNFDKLYIWGIHHPSTNQEQTSLYVQASGRVTVSTRRSQQTIIPNIGSRPWVRGLSSRISIYWTIVKPGDVLVINSNGNLIAPRGYFKMRTGKSSIMRSDAPIDTCISECITPNGSIPNDKPFQNVNKITYGACPKYVKQNTLKLATGMRNVPEKQTRGLFGAIAGFIENGWEGMIDGWYGFRHQNSEGTGQAADLKSTQAAIDQINGKLNRVIEKTNEKFHQIEKEFSEVEGRIQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTRRQLRENAEEMGNGCFKIYHKCDNACIESIRNGTYDHDVYRDEALNNRFQIKGVELKSGYKDWILWISFAISCFLLCVVLLGFIMWACQRGNIRCNICI"
my_seq_N1 = "MNPNQKIITIGSISIAIGIISLMLQIGNIISIWASHSIQTGSQNHTGICNQRIITYENSTWVNHTYVNINNTNVVAGKDKTSVTLAGNSSLCSISGWAIYTKDNSIRIGSKGDVFVIREPFISCSHLECRTFFLTQGALLNDKHSNGTVKDRSPYRALMSCPLGEAPSPYNSKFESVAWSASACHDGMGWLTIGISGPDNGAVAVLKYNGIITETIKSWKKQILRTQESECVCMNGSCFTIMTDGPSNGAASYKIFKIEKGKVTKSIELNAPNFHYEECSCYPDTGTVMCVCRDNWHGSNRPWVSFNQNLDYQIGYICSGVFGDNPRPKDGEGSCNPVTVDGANGVKGFSYKYGNGVWIGRTKSNRLRKGFEMIWDPNGWTNTDSDFSVKQDVVAITDWSGYSGSFVQHPELTGLDCIRPCFWVELVRGLPRENTTIWTSGSSISFCGVNSDTANWSWPDGAELPFTIDK"
my_seq_HIV1_A = "MRVKGIQMNSLRWGMLGWVTVYYGVPVWKDAETTLFCASDAKAYDAEVHNIWATHACVPTDPNPQEINLNVTEEFNMWKNNMVEQMHTDIISLWDQGLKPCVKLTPLCVTLDCTNCSYNVTKVSSLFYKLDVVQYRLINCNTSAITQACPKVTFEPIPIHYCAPAGFAILKCKDEKFNGTGLCKNVSTVQCTHGIKPVVSTQLLLNGSLAEEVRIRSENITNNAKNIIVQLASPVTINCIRPNNNTRGPGAYIIGEIRQAHCNVSEWNSTLQKVANQLFSGGDLEITTHSFNCGGEFFYCNTSGLFLQCRIKQIINMWQRAGQAIYAPPIPGVIRCKSNITGLILTRDETFRPGGGDMRDNWRSELYKYKVVKIEPIGVAPTRAKRRVVEREKRAIGAVFIGFLGAAGSTMGAASVTLTVQARQLLSGIVQQQSNLLRAIEAQQHLLKLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTNVPWNSSWSNKSEIWNMTWLQWDKEVSNYTQIIYTLIEESQNQQEKNEQDLLALDKWASLWNWFNISQWLWYIKIFIIIVGGLIGLRIVFAVLSVINRVRQGYSPLSFQTPPGLDRPGRIEEEGGEQDRGRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIATRTVEGWESLKYLGNLLVYWGRELKISAINLCDTIAIAVAGWTDRVIELGQRLCRAIHIPRRIRQGFERALL"
my_seq_HIV1_B = "MRVKGIRKNYWRWGLLGWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVPTDPNPQEVVLNVTENFNMWKNNMVEQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNITKEYALFYKLDVVPYRLISCNTSVITQACPKVSFEPIPIHYCAPAGFAILKCNDKKFNGTGPCTNVSTVQCTHGIRPVVSTQLLLNGSLAEEVVIRSENFTDNAKTIIVQLNESVEINCTRPNNNTRGPGAYIIGDIRQAHCNISKWNNTLKQIVKKLFQGGDPEIVMHSFNCGGEFFYCNTTQLFLPCRIKQIINMWQEVGKAMYAPPIRGQIRCSSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVKIEPLGVAPTKAKRRVVQREKRAIGAMFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTAVPWNTSWSNKSEIWNMTWMQWEREIDNYTGLIYTLIEESQNQQEKNEQELLELDKWASLWNWFDITNWLWYIKIFIMIVGGLIGLRIVFAVLSIVNRVRQGYSPLSFQTPPRPDRPEGIEEEGGERDRDRSGRLVDGFLALIWVDLRSLCLFSYHRLRDLLLIVARIVEGWEALKYWWNLLQYWSQELKNSAVSLLNATAIAVAEGTDRIIEVVQRAFRAIHIPRRIRQGLERALL"
my_seq_HIV1_C = "MRVRGILRNWWIWGILGWVTVYYGVPVWKEAKTTLFCASDAKAYEKEVHNVWATHACVPTDPNPQEMVLNVTENFNMWKNDMVDQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNTTKEYALFYRLDIVPYRLINCNTSTITQACPKVSFDPIPIHYCAPAGYAILKCNNKTFNGTGPCNNVSTVQCTHGIKPVVSTQLLLNGSLAEEIIIRSENLTNNAKTIIVHLNESVEIVCTRPNNNTRGPGTYIIGDIRQAHCNISKWNKTLQRVSKKLFTGGDLEITTHSFNCRGEFFYCNTSKLFLPCRIKQIINMWQEVGRAMYAPPIAGNITCKSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVEIKPLGIAPTKAKRRVVEREKRAIGAVFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQSNLLRAIEAQQHMLQLTVWGIKQLQARVLAIERYLKDQQLLGIWGCSGKLICTTAVPWNSSWSNKSDIWNMTWMQWDREISNYTDTIYRLLEESQNQQEKNEKDLLALDSWNNLWNWFDITNWLWYIKIFIMIVGGLIGLRIIFAVLSIVNRVRQGYSPLSFQTPPRPDRLGRIEEEGGEQDRDRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIAARAVEGWEALKYLGSLVQYWGLELKKSAISLLDTIAIAVAEGTDRIIELIQRICRAINIPRRIRQGFEAALL"

my_seqs = [my_seq_N2, my_seq_H1, my_seq_H3, my_seq_N1,
           my_seq_HIV1_A, my_seq_HIV1_B, my_seq_HIV1_C]
keys = ["1NN2_N2", "1RUZ_H1", "2VIU_H3", "3BEQ_N1", "6B0N_HIV_A", "6B0N_HIV_B",
       "6MYY_HIV1_C"]
# The files with p-values, output from our tool and filtered with filter_p_values.py
p_values = ["../../n2_all_prot_pvalues_output_corrected_multiple_testing_0.11_filtered.tsv",
            "../../h1_all_prot_pvalue_output_corrected_multiple_testing_0.11_filtered.tsv",
            "../../h3_all_prot_pvalues_output_corrected_multiple_testing_0.11_filtered.tsv",
            "../../n1_all_prot_pvalues_output_corrected_multiple_testing_0.11_filtered.tsv", 
            "../../hiv1_extant_subclass_a_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "../../hiv1_extant_subclass_b_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "../../hiv1_extant_subclass_c_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv"]

# initializeing the pickled dict
pickled_info = dict()
for k in keys:
    pickled_info[k] = dict()



In [ ]:
# For 1NN2 (N2)
parser = PDBParser()
current_seq = 0  # this index is then using with structures, my_seqs, p_values
structure = parser.get_structure(structures[current_seq], structures[current_seq])  # parsing structure

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]

# Aligning the sequence to chain A (Asymmetric unit)
alignments = pairwise2.align.globaldx(my_seqs[current_seq], structure_info[0]["A"]["seq"], blosum62)
# Putting all residues in a list
all_residues = structure_info[0]["A"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                            protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))

# for each pair, we calculate the distance accross models and keep the smallest one
chain = "A"
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):  # checking if that position is present in the structure
        distance = 2000
        for model in structure_info.values():
            c_dist = structure_info[0][chain]["residues"][matches[pos1][2]]["CA"] - model[chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance

epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", [x.get_id()[1] for x in epi_residues])
# getting all pairwise distances
all_distances = dict()
for residue1 in structure_info[0][chain]["residues"]:
    distance = 2000
    for model in structure_info.values():
        for residue2 in model[chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = distance

key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 1RUZ (H1)
parser = PDBParser()
current_seq = 1
structure = parser.get_structure(structures[current_seq], structures[current_seq])

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]
                
# we need to merge some chains because it's the sequence is split into different chains
# chains HI, JK, and LM have the same sequence
merging = ["HI", "JK", "LM"]
for k in merging:
    structure_info[0][k] = dict()
    structure_info[0][k]["seq"] = structure_info[0][k[0]]["seq"] + structure_info[0][k[1]]["seq"]
    structure_info[0][k]["residues"] = structure_info[0][k[0]]["residues"] + structure_info[0][k[1]]["residues"]


alignments = pairwise2.align.globaldx(my_seqs[current_seq], structure_info[0]["HI"]["seq"], blosum62)
all_residues = structure_info[0]["HI"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            

# There's only 1 model in this structure
# but we need to check the distances between HI-JK and HI-LM and take the smaller
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        distance = 2000
        for chain in ["HI", "JK", "LM"]:
            c_dist = structure_info[0]["HI"]["residues"][matches[pos1][2]]["CA"] - structure_info[0][chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance
 
epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", [x.get_id()[1] for x in epi_residues])

all_distances = dict()
for residue1 in structure_info[0]["HI"]["residues"]:
    distance = 2000
    for chain in ["HI", "JK", "LM"]:
        for residue2 in structure_info[0][chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = distance

                
key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 2VIU (H3)
parser = PDBParser()
current_seq = 2
structure = parser.get_structure(structures[current_seq], structures[current_seq])

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]
                
# Mergin chains A and B then aligning to that
for model in structure_info.keys():
    structure_info[model]["AB"] = dict()
    structure_info[model]["AB"]["seq"] = structure_info[model]["A"]["seq"] + structure_info[model]["B"]["seq"]
    structure_info[model]["AB"]["residues"] = structure_info[model]["A"]["residues"] + structure_info[model]["B"]["residues"]


alignments = pairwise2.align.globaldx(my_seqs[current_seq], structure_info[0]["AB"]["seq"], blosum62)
all_residues = structure_info[0]["AB"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            

# we need to check the distances accross models between residues in AB
chain = "AB"
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        distance = 2000
        for model in structure_info.values():
            c_dist = structure_info[0][chain]["residues"][matches[pos1][2]]["CA"] - model[chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance
        
epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", [x.get_id()[1] for x in epi_residues])

all_distances = dict()
for residue1 in structure_info[0][chain]["residues"]:
    distance = 2000
    for model in structure_info.values():
        for residue2 in model[chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                all_distances[(residue1._id[1], residue2._id[1])] = residue1["CA"] - residue2["CA"]

    
key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 3BEQ (N1)
parser = PDBParser()
current_seq = 3
structure = parser.get_structure(structures[current_seq], structures[current_seq])

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]


# I need to consider both chains A and B, but they are the same so I can align to only one
alignments = pairwise2.align.globaldx(my_seqs[current_seq], structure_info[0]["A"]["seq"], blosum62)
# However all_residues will be different for A and B
all_residues = structure_info[0]["A"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            
# There will be 4 comparisons, between A and A in first, A and B in the first
# between A and A in the second model and between A and B in the second model
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        # now only in first model
        distance = 2000
        for chain in ["A", "B"]:
            c_dist = structure_info[0]["A"]["residues"][matches[pos1][2]]["CA"] - structure_info[0][chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
                
        epi_distances[(pos1, pos2)] = distance

epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", [x.get_id()[1] for x in epi_residues])
# checking the distance between A in first model with A and B in second model
for pos1, pos2 in epi_distances.keys():
    for chain in ["A", "B"]:
        c_dist = structure_info[0]["A"]["residues"][matches[pos1][2]]["CA"] - structure_info[1][chain]["residues"][matches[pos2][2]]["CA"]
        if c_dist < epi_distances[(pos1, pos2)]:
            epi_distances[(pos1, pos2)] = c_dist
            

# distances between A and B in first model
all_distances = dict()
for residue1 in structure_info[0]["A"]["residues"]:
    distance = 2000
    for chain in ["A", "B"]:
        for residue2 in structure_info[0][chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = residue1["CA"] - residue2["CA"]

counter = 0
# distances between A and A,B in second model
for residue1 in structure_info[0]["A"]["residues"]:
    for chain in ["A", "B"]:
        for residue2 in structure_info[1][chain]["residues"]:
            distance = residue1["CA"] - residue2["CA"]
            try:
                if distance < all_distances[(residue1._id[1], residue2._id[1])]:
                    all_distances[(residue1._id[1], residue2._id[1])] = distance
            except:
                counter += 1

key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 6B0N (HIV1 Subtype A and B)
parser = PDBParser()
current_seq = 4
# The same script can be used for Subtype B, just changing the current_seq to 5
structure = parser.get_structure(structures[current_seq], structures[current_seq])

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]


# This structure has 3 models and I need to align to chain G
alignments = pairwise2.align.globaldx(my_seqs[current_seq],
                                      structure_info[0]["G"]["seq"], blosum62)

all_residues = structure_info[0]["G"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            

# for each pair, we calculate the distance accross models and keep the smallest one
chain = "G"
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        distance = 2000
        for model in structure_info.values():
            # comparing between first model and the other models and taking the minimum
            c_dist = structure_info[0][chain]["residues"][matches[pos1][2]]["CA"] - model[chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance

        
epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", "+".join([str(x.get_id()[1]) for x in epi_residues]))
# getting all pairwise distances
all_distances = dict()
for residue1 in structure_info[0][chain]["residues"]:
    distance = 2000
    for model in structure_info.values():
        for residue2 in model[chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = distance


key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 6B0N (HIV1 Subtype A and B)
parser = PDBParser()
current_seq = 5
# The same script can be used for Subtype B, just changing the current_seq to 5
structure = parser.get_structure(structures[current_seq], structures[current_seq])

# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]


# need to align to chain G
alignments = pairwise2.align.globaldx(my_seqs[current_seq],
                                      structure_info[0]["G"]["seq"], blosum62)

all_residues = structure_info[0]["G"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            

# for each pair, we calculate the distance accross models and keep the smallest one
chain = "G"
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        distance = 2000
        for model in structure_info.values():
#             assert matches[pos1][1] == protein_letters_3to1[model[chain]["residues"][matches[pos1][2]].resname]
            c_dist = structure_info[0][chain]["residues"][matches[pos1][2]]["CA"] - model[chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance

epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", "+".join([str(x.get_id()[1]) for x in epi_residues]))
# getting all pairwise distances
all_distances = dict()
for residue1 in structure_info[0][chain]["residues"]:
    distance = 2000
    for model in structure_info.values():
        for residue2 in model[chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = distance


key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# For 6MYY (HIV1 Subtype C)
parser = PDBParser()
current_seq = 6
# The same script can be used for Subtype B, just changing the current_seq to 5
structure = parser.get_structure(structures[current_seq], structures[current_seq])
# generating a dict with structure info (model_id:model) and inside each model we have (chain_id:chain)
structure_info = dict()
for model in structure:
    model_id = model.get_full_id()[1]
    structure_info[model_id] = dict()
    for chain in model.get_chains():
        chain_id = chain.get_full_id()[2]
        structure_info[model_id][chain_id] = {"residues":[], "seq":""}
        for residue in chain.get_residues():
            if residue.get_id()[0] == " ":
                structure_info[model_id][chain_id]["residues"].append(residue)
                structure_info[model_id][chain_id]["seq"] += protein_letters_3to1[residue.resname]

    
# aligning to A, because A, B, and E are the same
alignments = pairwise2.align.globaldx(my_seqs[current_seq], structure_info[0]["A"]["seq"], blosum62)

# However all_residues will be different for A and B
all_residues = structure_info[0]["A"]["residues"]

# matches here is a dictionary that matches a certain aa in the sequence to a residue in the structure
# this is needed because the residue ids in the structure is not the same as the cannonical sequence
# and this mapping is needed
alignment = alignments[0]
seq_pos = -1
residue_pos = -1
matches = dict()
for idx in range(len(alignment[0])):
    if alignment[0][idx] != "-":
        seq_pos += 1
    if alignment[1][idx] != "-":
        residue_pos += 1
    if (alignment[0][idx] != "-") and (alignment[1][idx] != "-"):
        assert alignment[0][idx] == my_seqs[current_seq][seq_pos]
        matches[seq_pos] = (seq_pos, alignment[0][idx], residue_pos,
                        protein_letters_3to1[all_residues[residue_pos].resname],
                            all_residues[residue_pos])

        
# Loading the epistatic pairs
epi_pairs = []
with open(p_values[current_seq], "r") as in_file:
    next(in_file)
    for l in in_file:
        l = l.split("\t")
        # first and second column of that table
        pos1 = int(l[0])
        pos2 = int(l[1])
        if pos1 > pos2:
            epi_pairs.append((pos1, pos2))
        else:
            epi_pairs.append((pos2, pos1))
            
# for each pair we calculate the distance between A
epi_distances = dict()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        distance = 2000
        for chain in ["A", "B", "E"]:
#         for chain in structure_info.values():
#             assert matches[pos1][1] == protein_letters_3to1[model[chain]["residues"][matches[pos1][2]].resname]
            c_dist = structure_info[0]["A"]["residues"][matches[pos1][2]]["CA"] - structure_info[0][chain]["residues"][matches[pos2][2]]["CA"]
            if c_dist < distance:
                distance = c_dist
        epi_distances[(pos1, pos2)] = distance

epi_residues = set()
for pos1, pos2 in epi_pairs:
    if (pos1 in matches) and (pos2 in matches):
        epi_residues.add(matches[pos1][4])
        epi_residues.add(matches[pos2][4])
print("epistatic residues are", "+".join([str(x.get_id()[1]) for x in epi_residues]))

all_distances = dict()
for residue1 in structure_info[0]["A"]["residues"]:
    distance = 2000
    for chain in ["A", "B", "E"]:
        for residue2 in structure_info[0][chain]["residues"]:
            if residue1["CA"] - residue2["CA"] < distance:
                distance = residue1["CA"] - residue2["CA"]
                all_distances[(residue1._id[1], residue2._id[1])] = distance

key = keys[current_seq]
pickled_info[key]["epi_pairs"] = epi_pairs
pickled_info[key]["epi_distances"] = epi_distances
pickled_info[key]["all_distances"] = all_distances

wilcoxon_R(epi_distances, all_distances, key)

In [ ]:
# overlapping the epistatic positions between hiv1 a,b, and c, to see if there are there
# are the same pairs in all three or the pairs are different.
# the sequences have the same length without gaps because it's the consensus sequence
all_epi_pairs = []
for i in range(4, 7):
    all_epi_pairs.append(set())
    with open(p_values[i], "r") as in_file:
        next(in_file)
        for l in in_file:
            l = l.split("\t")
            # first and second column of that table
            pos1 = int(l[0])
            pos2 = int(l[1])
            # to store them in the same order for comparison, in case they were flipped
            if pos1 > pos2:
                all_epi_pairs[i-4].add((pos1, pos2))
            else:
                all_epi_pairs[i-4].add((pos2, pos1))
            

In [ ]:
# to pring the overlap between the pairs 
print(len(all_epi_pairs[0]), len(all_epi_pairs[1]), len(all_epi_pairs[2]))
print(all_epi_pairs[0].intersection(all_epi_pairs[1]))
print(all_epi_pairs[0].intersection(all_epi_pairs[2]))
print(all_epi_pairs[1].intersection(all_epi_pairs[2]))

In [ ]:
# outputting the dictionary as a pickled dict
with open("epi_distance_information.pickle", "wb") as handle:
    pickle.dump(pickled_info, handle)


In [ ]:
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)
    
def make_boxplot(distance_data, output_loc):
    labels = list(distance_data.keys())
    epi_distances = [list(x['epi_distances'].values()) for x in distance_data.values()]
    epi_distances_bxp = plt.boxplot(epi_distances, positions=[(x*2)+4.0-0.4 for x in range(len(labels))], 
                      sym='', widths=0.6)
    all_distances = [list(x['all_distances'].values()) for x in distance_data.values()]
    all_distances_bxp = plt.boxplot(all_distances, positions=[(x*2)+4.0+0.4 for x in range(len(labels))], 
                      sym='', widths=0.6)
    set_box_color(epi_distances_bxp, 'red') # colors are from http://colorbrewer2.org/
    set_box_color(all_distances_bxp, 'blue')

    plt.plot([], c='red', label='Epistatic pairs')
    plt.plot([], c='blue', label='All pairs')
    plt.xticks([(x*2)+4 for x in range(len(labels))], labels, rotation=-45)
    plt.legend(loc='upper right')
    plt.title("Distance distribution between residues in 3D structure")
#     plt.show()
    plt.savefig(output_loc, dpi=1000, bbox_inches='tight')

def violin_color(vplot, facecolor, edgecolor):
    for pc in vplot['bodies']:
        pc.set_facecolor(facecolor)
        pc.set_edgecolor(edgecolor)
        
def make_violins(distance_data, output_loc):
    fig, ax = plt.subplots()
    labels = list(distance_data.keys())
    epi_distances = [list(x['epi_distances'].values()) for x in distance_data.values()]
    all_distances = [list(x['all_distances'].values()) for x in distance_data.values()]
    
    violin_epi = ax.violinplot(epi_distances, positions=[(x*2)+3.0-0.4 for x in range(len(labels))])
    violin_all = ax.violinplot(all_distances, positions=[(x*2)+3.0+0.4 for x in range(len(labels))])
    violin_color(violin_epi, "red", "red")
    violin_color(violin_all, "blue", "blue")
    plt.title("Distance distribution between residues in 3D structure")
    
    plt.xticks([(x*2)+3 for x in range(len(labels))], labels, rotation=-45)
    plt.savefig(output_loc, dpi=1000, bbox_inches='tight')
    
with open("epi_distance_information.pickle", "rb") as infile:
    data = pickle.load(infile)
make_boxplot(data, "boxplot.png")
make_violins(data, "test_violins.png")

In [ ]:
# this snippet was used to map our hiv sequences to the table with HXB2 HIV1 
# info table with explanations for each position
info_table = []  
with open("../../ann_table.tsv", "r") as infile:  
    for l in infile:
        info_table.append([x for x in l.strip().split("\t") if x != ""])  

# printing the number of residues with some annotation
all_n_annotations = len([x for x in info_table if len(x) > 3])
all_n_no_annotations = len([x for x in info_table if len(x) <= 3])
print(f"num of aa with annotations {all_n_annotations}, num of aa without annotations {all_n_no_annotations}, percentage {(all_n_annotations*100)/(all_n_annotations + all_n_no_annotations)}")
    
def get_info(hiv_seq, pairs_file, table): 
    ann_seq = "MRVKEKYQHLWRWGWRWGTMLLGMLMICSATEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVPTDPNPQEVVLVNVTENFNMWKNDMVEQMHEDIISLWDQSLKPCVKLTPLCVSLKCTDLKNDTNTNSSSGRMIMEKGEIKNCSFNISTSIRGKVQKEYAFFYKLDIIPIDNDTTSYKLTSCNTSVITQACPKVSFEPIPIHYCAPAGFAILKCNNKTFNGTGPCTNVSTVQCTHGIRPVVSTQLLLNGSLAEEEVVIRSVNFTDNAKTIIVQLNTSVEINCTRPNNNTRKRIRIQRGPGRAFVTIGKIGNMRQAHCNISRAKWNNTLKQIASKLREQFGNNKTIIFKQSSGGDPEIVTHSFNCGGEFFYCNSTQLFNSTWFNSTWSTEGSNNTEGSDTITLPCRIKQIINMWQKVGKAMYAPPISGQIRCSSNITGLLLTRDGGNSNNESEIFRPGGGDMRDNWRSELYKYKVVKIEPLGVAPTKAKRRVVQREKRAVGIGALFLGFLGAAGSTMGAASMTLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARILAVERYLKDQQLLGIWGCSGKLICTTAVPWNASWSNKSLEQIWNHTTWMEWDREINNYTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWFNITNWLWYIKLFIMIVGGLVGLRIVFAVLSIVNRVRQGYSPLSFQTHLPTPRGPDRPEGIEEEGGERDRDRSIRLVNGSLALIWDDLRSLCLFSYHRLRDLLLIVTRIVELLGRRGWEALKYWWNLLQYWSQELKNSAVSLLNATAIAVAEGTDRVIEVVQGACRAIRHIPRRIRQGLERILL"
    alignment = pairwise2.align.globalxx(hiv_seq, ann_seq)[0] 
    # Loading the epistatic pairs 
    epi_positions = set()
    with open(pairs_file, "r") as in_file: 
        next(in_file) 
        for l in in_file: 
            l = l.split("\t")
            # first and second column of that table 
            pos1 = int(l[0]) 
            pos2 = int(l[1])
            epi_positions.add(pos1)
            epi_positions.add(pos2)
#             if pos1 > pos2:
#                 epi_pairs.append((pos1, pos2)) 
#             else:
#                 epi_pairs.append((pos2, pos1)) 
    seq_pos = -1 
    ann_pos = -1 
    matches = dict() 
    for idx in range(len(alignment[0])): 
        if alignment[0][idx] != "-": 
            seq_pos += 1 
        if alignment[1][idx] != "-": 
            ann_pos += 1 
        if alignment[0][idx] != "-" and alignment[1][idx] != "-": 
            assert alignment[0][idx] == hiv_seq[seq_pos] 
            assert alignment[1][idx] == ann_seq[ann_pos] 
            matches[seq_pos] = table[ann_pos]
    
    epi_pairs_info = dict() 
    for pos in epi_positions:
        if pos in matches:
            tmp = deepcopy(matches[pos])
            tmp.append(hiv_seq[pos])
            epi_pairs_info[pos] = tmp
            
#         if pos1 not in epi_pairs_info: 
#             if pos1 in matches: 
#                 tmp = deepcopy(matches[pos1]) 
#                 tmp.append(hiv_seq[pos1]) 
#                 epi_pairs_info[pos1] = tmp 
#             if pos2 not in epi_pairs_info: 
#                 if pos2 in matches: 
#                     tmp = deepcopy(matches[pos2]) 
#                     tmp.append(hiv_seq[pos2]) 
#                     epi_pairs_info[pos2] = tmp 
    return epi_pairs_info

my_seq_HIV1_A = "MRVKGIQMNSLRWGMLGWVTVYYGVPVWKDAETTLFCASDAKAYDAEVHNIWATHACVPTDPNPQEINLNVTEEFNMWKNNMVEQMHTDIISLWDQGLKPCVKLTPLCVTLDCTNCSYNVTKVSSLFYKLDVVQYRLINCNTSAITQACPKVTFEPIPIHYCAPAGFAILKCKDEKFNGTGLCKNVSTVQCTHGIKPVVSTQLLLNGSLAEEVRIRSENITNNAKNIIVQLASPVTINCIRPNNNTRGPGAYIIGEIRQAHCNVSEWNSTLQKVANQLFSGGDLEITTHSFNCGGEFFYCNTSGLFLQCRIKQIINMWQRAGQAIYAPPIPGVIRCKSNITGLILTRDETFRPGGGDMRDNWRSELYKYKVVKIEPIGVAPTRAKRRVVEREKRAIGAVFIGFLGAAGSTMGAASVTLTVQARQLLSGIVQQQSNLLRAIEAQQHLLKLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTNVPWNSSWSNKSEIWNMTWLQWDKEVSNYTQIIYTLIEESQNQQEKNEQDLLALDKWASLWNWFNISQWLWYIKIFIIIVGGLIGLRIVFAVLSVINRVRQGYSPLSFQTPPGLDRPGRIEEEGGEQDRGRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIATRTVEGWESLKYLGNLLVYWGRELKISAINLCDTIAIAVAGWTDRVIELGQRLCRAIHIPRRIRQGFERALL"
my_seq_HIV1_B = "MRVKGIRKNYWRWGLLGWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVPTDPNPQEVVLNVTENFNMWKNNMVEQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNITKEYALFYKLDVVPYRLISCNTSVITQACPKVSFEPIPIHYCAPAGFAILKCNDKKFNGTGPCTNVSTVQCTHGIRPVVSTQLLLNGSLAEEVVIRSENFTDNAKTIIVQLNESVEINCTRPNNNTRGPGAYIIGDIRQAHCNISKWNNTLKQIVKKLFQGGDPEIVMHSFNCGGEFFYCNTTQLFLPCRIKQIINMWQEVGKAMYAPPIRGQIRCSSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVKIEPLGVAPTKAKRRVVQREKRAIGAMFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTAVPWNTSWSNKSEIWNMTWMQWEREIDNYTGLIYTLIEESQNQQEKNEQELLELDKWASLWNWFDITNWLWYIKIFIMIVGGLIGLRIVFAVLSIVNRVRQGYSPLSFQTPPRPDRPEGIEEEGGERDRDRSGRLVDGFLALIWVDLRSLCLFSYHRLRDLLLIVARIVEGWEALKYWWNLLQYWSQELKNSAVSLLNATAIAVAEGTDRIIEVVQRAFRAIHIPRRIRQGLERALL"
my_seq_HIV1_C = "MRVRGILRNWWIWGILGWVTVYYGVPVWKEAKTTLFCASDAKAYEKEVHNVWATHACVPTDPNPQEMVLNVTENFNMWKNDMVDQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNTTKEYALFYRLDIVPYRLINCNTSTITQACPKVSFDPIPIHYCAPAGYAILKCNNKTFNGTGPCNNVSTVQCTHGIKPVVSTQLLLNGSLAEEIIIRSENLTNNAKTIIVHLNESVEIVCTRPNNNTRGPGTYIIGDIRQAHCNISKWNKTLQRVSKKLFTGGDLEITTHSFNCRGEFFYCNTSKLFLPCRIKQIINMWQEVGRAMYAPPIAGNITCKSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVEIKPLGIAPTKAKRRVVEREKRAIGAVFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQSNLLRAIEAQQHMLQLTVWGIKQLQARVLAIERYLKDQQLLGIWGCSGKLICTTAVPWNSSWSNKSDIWNMTWMQWDREISNYTDTIYRLLEESQNQQEKNEKDLLALDSWNNLWNWFDITNWLWYIKIFIMIVGGLIGLRIIFAVLSIVNRVRQGYSPLSFQTPPRPDRLGRIEEEGGEQDRDRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIAARAVEGWEALKYLGSLVQYWGLELKKSAISLLDTIAIAVAEGTDRIIELIQRICRAINIPRRIRQGFEAALL"

proteins = {"HIV1 subtype A":my_seq_HIV1_A,
            "HIV1 subtype B":my_seq_HIV1_B,
            "HIV1 subtype C":my_seq_HIV1_C,
            "HIV Env Protein": "NA"}

files = {"HIV1 subtype A":"../../hiv1_extant_subclass_a_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "HIV1 subtype B":"../../hiv1_extant_subclass_b_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "HIV1 subtype C":"../../hiv1_extant_subclass_c_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv"}

results = []
for p_name, p in proteins.items():
    if p_name == "HIV Env Protein":  # skip this one
        continue
        
    epi_pairs_info = get_info(p, files[p_name], info_table)

    markers = {"Core":0, "Disorder":0, "Ion Interaction":0, "Ligand Interaction":0,
               "Metal Interaction":0, "Protein Interaction":0, "Surface":0}
    for key in markers:
        for pos in epi_pairs_info.keys():
            if key in epi_pairs_info[pos]:
                markers[key] += 1
#                 epi_pairs_info[pos].remove(key)

    n_with_annotations = len([x for x in epi_pairs_info.values() if len(x) > 4])
    print(f"n of epi positions with annotation {n_with_annotations}")
    n_without_annotations = len([x for x in epi_pairs_info.values() if len(x) <= 4])
    print(f"n of epi positions without annotation {n_without_annotations}")
    print(p_name, (n_with_annotations *100)/(n_with_annotations + n_without_annotations))
    fisher_test = [[n_with_annotations, n_without_annotations], [all_n_annotations, all_n_no_annotations]]
    print(f"fisher exact test results {print(stats.fisher_exact(fisher_test, alternative='greater'))}")
    results.append((epi_pairs_info, markers))
    
# adding markers for all residues
markers = {"Core":0, "Disorder":0, "Ion Interaction":0, "Ligand Interaction":0,
           "Metal Interaction":0, "Protein Interaction":0, "Surface":0}
for i in info_table:
    if i[-1] in markers:
        markers[i[-1]] += 1
results.append(("NA", markers))

ticks = []
for key in markers.keys():
    ticks.append(key)
    
heights = [[], [], [], []]
for idx, p_name in enumerate(proteins.keys()):
    for value in results[idx][1].values():
        heights[idx].append(value)
        
# turning them into fractions
for j in range(len(heights)):
    for idx, i in enumerate(heights[j]):
        heights[j][idx] = float(i/sum(heights[j]))
        
width = 0.35
spacing1 = [(x*2) + 1 for x in range(len(markers))]
spacing2 = [x + width for x in spacing1]
spacing3 = [x + width for x in spacing2]
spacing4 = [x + width for x in spacing3]

print(heights)
p1 = plt.bar(spacing1, heights[0], width)
p2 = plt.bar(spacing2, heights[1], width)
p3 = plt.bar(spacing3, heights[2], width)
p4 = plt.bar(spacing4, heights[3], width)

plt.ylabel('Number of residues')
plt.title('Distribution of detected positions and their class in protein')
assert len(spacing1) == len(ticks)
spacing2[1] += width
spacing2[2] += 2*width
spacing2[3] += 1
spacing2[4] += 1
spacing2[5] += 1
spacing2[6] += width

plt.xticks(spacing2, ticks, rotation=-45)
plt.tick_params(axis='x', length=0)
plt.legend((p1[0], p2[0], p3[0], p4[0]), ('HIV1 subtype A', 'HIV1 subtype B', 'HIV1 subtype C', 'HIV1 Env protein HXB2'))

plt.savefig("dist_of_classes.png", dpi=800, bbox_inches='tight')

In [ ]:
# this snippet was used to map our hiv sequences to the table with HXB2 HIV1 
# info table with explanations for each position
info_table = []  
with open("../../ann_table.tsv", "r") as infile:  
    for l in infile:  
        info_table.append([x for x in l.strip().split("\t") if x != ""])  

def get_info(hiv_seq, pairs_file, table): 
    ann_seq = "MRVKEKYQHLWRWGWRWGTMLLGMLMICSATEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVPTDPNPQEVVLVNVTENFNMWKNDMVEQMHEDIISLWDQSLKPCVKLTPLCVSLKCTDLKNDTNTNSSSGRMIMEKGEIKNCSFNISTSIRGKVQKEYAFFYKLDIIPIDNDTTSYKLTSCNTSVITQACPKVSFEPIPIHYCAPAGFAILKCNNKTFNGTGPCTNVSTVQCTHGIRPVVSTQLLLNGSLAEEEVVIRSVNFTDNAKTIIVQLNTSVEINCTRPNNNTRKRIRIQRGPGRAFVTIGKIGNMRQAHCNISRAKWNNTLKQIASKLREQFGNNKTIIFKQSSGGDPEIVTHSFNCGGEFFYCNSTQLFNSTWFNSTWSTEGSNNTEGSDTITLPCRIKQIINMWQKVGKAMYAPPISGQIRCSSNITGLLLTRDGGNSNNESEIFRPGGGDMRDNWRSELYKYKVVKIEPLGVAPTKAKRRVVQREKRAVGIGALFLGFLGAAGSTMGAASMTLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARILAVERYLKDQQLLGIWGCSGKLICTTAVPWNASWSNKSLEQIWNHTTWMEWDREINNYTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWFNITNWLWYIKLFIMIVGGLVGLRIVFAVLSIVNRVRQGYSPLSFQTHLPTPRGPDRPEGIEEEGGERDRDRSIRLVNGSLALIWDDLRSLCLFSYHRLRDLLLIVTRIVELLGRRGWEALKYWWNLLQYWSQELKNSAVSLLNATAIAVAEGTDRVIEVVQGACRAIRHIPRRIRQGLERILL"
    alignment = pairwise2.align.globalxx(hiv_seq, ann_seq)[0] 
    # Loading the epistatic pairs 
    epi_positions = set()
    with open(pairs_file, "r") as in_file: 
        next(in_file) 
        for l in in_file: 
            l = l.split("\t") 
            # first and second column of that table 
            pos1 = int(l[0]) 
            pos2 = int(l[1])
            epi_positions.add(pos1)
            epi_positions.add(pos2)
#             if pos1 > pos2: 
#                 epi_pairs.append((pos1, pos2)) 
#             else: 
#                 epi_pairs.append((pos2, pos1)) 
    seq_pos = -1 
    ann_pos = -1 
    matches = dict() 
    for idx in range(len(alignment[0])): 
        if alignment[0][idx] != "-": 
            seq_pos += 1 
        if alignment[1][idx] != "-": 
            ann_pos += 1 
        if alignment[0][idx] != "-" and alignment[1][idx] != "-": 
            assert alignment[0][idx] == hiv_seq[seq_pos] 
            assert alignment[1][idx] == ann_seq[ann_pos] 
            matches[seq_pos] = table[ann_pos]

    return matches

my_seq_HIV1_A = "MRVKGIQMNSLRWGMLGWVTVYYGVPVWKDAETTLFCASDAKAYDAEVHNIWATHACVPTDPNPQEINLNVTEEFNMWKNNMVEQMHTDIISLWDQGLKPCVKLTPLCVTLDCTNCSYNVTKVSSLFYKLDVVQYRLINCNTSAITQACPKVTFEPIPIHYCAPAGFAILKCKDEKFNGTGLCKNVSTVQCTHGIKPVVSTQLLLNGSLAEEVRIRSENITNNAKNIIVQLASPVTINCIRPNNNTRGPGAYIIGEIRQAHCNVSEWNSTLQKVANQLFSGGDLEITTHSFNCGGEFFYCNTSGLFLQCRIKQIINMWQRAGQAIYAPPIPGVIRCKSNITGLILTRDETFRPGGGDMRDNWRSELYKYKVVKIEPIGVAPTRAKRRVVEREKRAIGAVFIGFLGAAGSTMGAASVTLTVQARQLLSGIVQQQSNLLRAIEAQQHLLKLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTNVPWNSSWSNKSEIWNMTWLQWDKEVSNYTQIIYTLIEESQNQQEKNEQDLLALDKWASLWNWFNISQWLWYIKIFIIIVGGLIGLRIVFAVLSVINRVRQGYSPLSFQTPPGLDRPGRIEEEGGEQDRGRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIATRTVEGWESLKYLGNLLVYWGRELKISAINLCDTIAIAVAGWTDRVIELGQRLCRAIHIPRRIRQGFERALL"
my_seq_HIV1_B = "MRVKGIRKNYWRWGLLGWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVPTDPNPQEVVLNVTENFNMWKNNMVEQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNITKEYALFYKLDVVPYRLISCNTSVITQACPKVSFEPIPIHYCAPAGFAILKCNDKKFNGTGPCTNVSTVQCTHGIRPVVSTQLLLNGSLAEEVVIRSENFTDNAKTIIVQLNESVEINCTRPNNNTRGPGAYIIGDIRQAHCNISKWNNTLKQIVKKLFQGGDPEIVMHSFNCGGEFFYCNTTQLFLPCRIKQIINMWQEVGKAMYAPPIRGQIRCSSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVKIEPLGVAPTKAKRRVVQREKRAIGAMFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQNNLLRAIEAQQHLLQLTVWGIKQLQARVLAVERYLKDQQLLGIWGCSGKLICTTAVPWNTSWSNKSEIWNMTWMQWEREIDNYTGLIYTLIEESQNQQEKNEQELLELDKWASLWNWFDITNWLWYIKIFIMIVGGLIGLRIVFAVLSIVNRVRQGYSPLSFQTPPRPDRPEGIEEEGGERDRDRSGRLVDGFLALIWVDLRSLCLFSYHRLRDLLLIVARIVEGWEALKYWWNLLQYWSQELKNSAVSLLNATAIAVAEGTDRIIEVVQRAFRAIHIPRRIRQGLERALL"
my_seq_HIV1_C = "MRVRGILRNWWIWGILGWVTVYYGVPVWKEAKTTLFCASDAKAYEKEVHNVWATHACVPTDPNPQEMVLNVTENFNMWKNDMVDQMHEDIISLWDQSLKPCVKLTPLCVTLNCKNCSFNTTKEYALFYRLDIVPYRLINCNTSTITQACPKVSFDPIPIHYCAPAGYAILKCNNKTFNGTGPCNNVSTVQCTHGIKPVVSTQLLLNGSLAEEIIIRSENLTNNAKTIIVHLNESVEIVCTRPNNNTRGPGTYIIGDIRQAHCNISKWNKTLQRVSKKLFTGGDLEITTHSFNCRGEFFYCNTSKLFLPCRIKQIINMWQEVGRAMYAPPIAGNITCKSNITGLLLTRDETFRPGGGDMRDNWRSELYKYKVVEIKPLGIAPTKAKRRVVEREKRAIGAVFLGFLGAAGSTMGAASITLTVQARQLLSGIVQQQSNLLRAIEAQQHMLQLTVWGIKQLQARVLAIERYLKDQQLLGIWGCSGKLICTTAVPWNSSWSNKSDIWNMTWMQWDREISNYTDTIYRLLEESQNQQEKNEKDLLALDSWNNLWNWFDITNWLWYIKIFIMIVGGLIGLRIIFAVLSIVNRVRQGYSPLSFQTPPRPDRLGRIEEEGGEQDRDRSIRLVSGFLALAWDDLRSLCLFSYHRLRDFILIAARAVEGWEALKYLGSLVQYWGLELKKSAISLLDTIAIAVAEGTDRIIELIQRICRAINIPRRIRQGFEAALL"

proteins = {"HIV1 subtype A":my_seq_HIV1_A,
            "HIV1 subtype B":my_seq_HIV1_B,
            "HIV1 subtype C":my_seq_HIV1_C}

files = {"HIV1 subtype A":"../../hiv1_extant_subclass_a_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "HIV1 subtype B":"../../hiv1_extant_subclass_b_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv",
            "HIV1 subtype C":"../../hiv1_extant_subclass_c_pvalues_output_corrected_multiple_testing_0.051_filtered.tsv"}

results = []
for p_name, p in proteins.items():
    epi_pairs_info = get_info(p, files[p_name], info_table)

    markers = {"Core":0, "Disorder":0, "Ion Interaction":0, "Ligand Interaction":0,
               "Metal Interaction":0, "Protein Interaction":0, "Surface":0}
    
    for key in markers:
        for pos in epi_pairs_info.keys():
            if key in epi_pairs_info[pos]:
                markers[key] += 1
#                 epi_pairs_info[pos].remove(key)
                
    print(markers)
    print(p_name, len([x for x in epi_pairs_info.values() if len(x) > 2])/len(epi_pairs_info))
    results.append((epi_pairs_info, markers))
    
 
ticks = []
for key in markers.keys():
    ticks.append(key)
    
heights = [[], [], []]
for idx, p_name in enumerate(proteins.keys()):
    for value in results[idx][1].values():
        heights[idx].append(value)
width = 0.35
spacing1 = [(x*2) + 1 for x in range(len(markers))]
spacing2 = [x + width for x in spacing1]
spacing3 = [x + width for x in spacing2]
print(heights)
p1 = plt.bar(spacing1, heights[0], width)
p2 = plt.bar(spacing2, heights[1], width)
p3 = plt.bar(spacing3, heights[2], width)
plt.ylabel('Number of residues')
plt.title('Distribution of detected positions and their class in protein')
assert len(spacing1) == len(ticks)
spacing2[1] += width
spacing2[2] += 2*width
spacing2[3] += 1
spacing2[4] += 1
spacing2[5] += 1
spacing2[6] += width

plt.xticks(spacing2, ticks, rotation=-45)
plt.tick_params(axis='x', length=0)
plt.legend((p1[0], p2[0], p3[0]), ('HIV1 subtype A', 'HIV1 subtype B', 'HIV1 subtype C'))

plt.savefig("dist_of_classes_all_residues.png", dpi=800, bbox_inches='tight')

In [ ]:
results[0][0]